In [1]:
print('Enter first polynomial')
a = str(input())
print('Enter the operators : -, + or *')
op = str(input())
print('Enter the second polynomial')
b = str(input())

Enter first polynomial
x^2 + 2*x*y
Enter the operators : -, + or *
*
Enter the second polynomial
x*y*z + 3*z


In [2]:
def decode_term(term): # simplifies terms : eg x^2 * z  = x*x*z
    tmp = ""
    l = len(term)
    counter = 0
    is_previous_integer = 0 # flag becomes 1 if previous term was an integer
    integral_val = 1 # all the integral terms are multiplied here
    tmpstr = "" # temporarily contains integral value
    while counter<l:
        if is_previous_integer==1:
            if str.isdigit(term[counter]):
                tmpstr+=term[counter]
            else: 
                is_previous_integer=0
                integral_val*=int(tmpstr)
                tmpstr=''
            counter+=1
            continue                
        if term[counter] == '*': 
            counter+=1
            continue
        if term[counter] == '^':
            counter+=1
            split = ''
            for j in range(1,int(term[counter])):
                tmp+=term[counter-2]
            counter+=1
        else: 
            if str.isdigit(term[counter]):
                tmpstr=term[counter]
                is_previous_integer = 1
            else:
                tmp+=term[counter]
            counter+=1
    return str(integral_val)+tmp
# example :
print('Test : Decoding 12*x^4*14*y^2*z^6',decode_term('12*x^4*14*y^2*z^6'))

def is_same(x,y): # checks if 2 unit of polynomials are same (so that both can be added)
    blank= ''
    x_decoded = sorted(decode_term(x))
    y_decoded = sorted(decode_term(y))
    l_x = len(x_decoded)
    l_y = len(y_decoded)
    # removing integer digits
    for i in range(l_x):
        if not str.isdigit(x_decoded[i]):
            x_decoded = x_decoded[i:]
            break
    for i in range(l_y):
        if not str.isdigit(y_decoded[i]):
            y_decoded = y_decoded[i:]
            break
    return x_decoded==y_decoded
print("Test : Checking for 12*x*y*z and 18*y*x*z",is_same('12*x*y*z','18*y*x*z'))

Test : Decoding 12*x^4*14*y^2*z^6 168xxxxyyzzzzzz
Test : Checking for 12*x*y*z and 18*y*x*z True


In [3]:
def addsub_term(x,y,op): # a and b are passed after checking that variables are same
                        # and they can be added
    x_decoded = sorted(decode_term(x))
    y_decoded = sorted(decode_term(y))
    l_x = len(x_decoded)
    l_y = len(y_decoded)
    digit1,digit2 = '',''
    # extracting integer digits
    for i in range(l_x):
        if not str.isdigit(x_decoded[i]):
            x_decoded = x_decoded[i:] # polynomial without integral term
            break
        else:
            digit1+= x_decoded[i]
    digit1 = int(digit1)
    for i in range(l_y):
        if not str.isdigit(y_decoded[i]):
            break
        else:
            digit2+= y_decoded[i]
    digit2 = int(digit2)
    sum_digit = 0
    if op=='+': sum_digit=digit1+digit2
    elif op=='-': sum_digit=digit1-digit2
    if sum_digit==0: return '0'
    return str(sum_digit)+"".join(x_decoded)
print("Test subtracting: 15*a*b*c, a*6*c*b, -: ",addsub_term('15*a*b*c','a*6*c*b','-'))

def multiply_term(a,b):
    return "".join(sorted(decode_term(a+b)))

print("Test multiplying: 2*a^2*b^2, 2*b*a*z",multiply_term('2*a^2*b^2','2*b*a*z'))

Test subtracting: 15*a*b*c, a*6*c*b, -:  9abc
Test multiplying: 2*a^2*b^2, 2*b*a*z 4aaabbbz


In [4]:
def invert(op): #returns - for + and + for -
    if op=='+': return '-'
    else: return '+'
def addsub(x,y,op):
    x_lis = x.split(' ')
    y_lis = y.split(' ')
    if op=='-':
        for i in range(1,len(y_lis),2): y_lis[i] = invert(y_lis[i])
    result_lis = x_lis
    result_lis.append(op)
    for i in y_lis:result_lis.append(i)
    for i in range(0,len(result_lis),2):
        if result_lis[i]=='':continue
        for j in range(i+2,len(result_lis),2):
            if result_lis[j]=='':continue
            if is_same(result_lis[i],result_lis[j]):
                result_lis[i] = addsub_term(result_lis[i],result_lis[j],result_lis[j-1])
                result_lis[j]=''
                result_lis[j-1] = ''
    # making list a string
    result = ''
    for i in range(0,len(result_lis)):
        cur = result_lis[i]
        l = len(cur)
        if l>0 and not l==1:
            if cur[0]=='-':
                cur=cur[1:]
                result=result[:-1]
                result+='-'
        if cur=='0':
            result=result[:-1]
            cur=''
        result+=cur
            
    return result


In [5]:
addsub(a,b,'-')

'x^2+2*x*y-x*y*z-3*z'

In [6]:
addsub('x^2','x*y - x^2','-')

'2xx-x*y'

In [7]:
def shorten(x): # xxx -> x^3
    num= ''
    for i in range(len(x)):
        if str.isdigit(x[i]):num+=x[i]
        else:break
    l = len(num)
    x=x[l:] # removing digits from x
    curindex = 0
    u = []
    for i in x: 
        if not i in u: u.append(i)
    tempcount = 0
    current_count = [0]*len(u)
    for i in x:
        if not i==u[curindex]: curindex+=1
        current_count[curindex] +=1
    for i in range(0,len(u)):
        if current_count[i]==1: num+=('*'+u[i])
        else: num+=('*'+u[i]+'^'+str(current_count[i]))
    if num[0]=='*': return num[1:]
    else: return num

In [8]:
shorten('15xxabyz')

'15*x^2*a*b*y*z'

In [9]:
def getSign(a,b):
    if a=='+':
        if b=='+':return '+'
        else: return '-'
    else:
        if b=='+':return '-'
        else: return '+'

def multiply(a,b):
    ans = []
    x = a.split(' ')
    y = b.split(' ')
    x_sign,y_sign = '+','+'
    for i in x:
        if i=='+' or i=='-':
            x_sign=i
            continue
        for j in y: 
            if j=='+' or j=='-':
                y_sign=j
                continue
            ans.append(getSign(x_sign,y_sign))
            term = shorten(multiply_term(i,j))
            ans.append(term)
    return "".join(ans)[1:]

In [10]:
print('Operation : (',a,') ',op,' (',b,')')
if op=='*':
    print('Result is: ',multiply(a,b))
elif op=='+' or op=='-':
    print('Result is: ',addsub(a,b,op))
else:print('Unknown operator')

Operation : ( x^2 + 2*x*y )  *  ( x*y*z + 3*z )
Result is:  1*x^3*y*z+3*x^2*z+2*x^2*y^2*z+6*x*y*z
